In [1]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [2]:
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
	return [randint(0, 99) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

In [3]:
# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(sequence)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = sequence.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
	return X, y

In [4]:
# prepare data for the LSTM
def get_data(n_in, n_out):
	# generate random sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to X,y pairs
	X,y = to_supervised(encoded, n_in, n_out)
	return X,y


In [5]:
# define LSTM
n_in = 5
n_out = 5
encoded_length = 100
batch_size = 7
model = Sequential()
model.add(LSTM(20, batch_input_shape=(batch_size, n_in, encoded_length), return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
# train LSTM
for epoch in range(500):
	# generate new random sequence
	X,y = get_data(n_in, n_out)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# evaluate LSTM
X,y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=0)
# decode all pairs
for i in range(len(X)):
	print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

Epoch 1/1
 - 5s - loss: 4.5942 - acc: 0.0381
Epoch 1/1
 - 0s - loss: 4.6024 - acc: 0.0476
Epoch 1/1
 - 0s - loss: 4.5952 - acc: 0.0095
Epoch 1/1
 - 0s - loss: 4.5977 - acc: 0.0095
Epoch 1/1
 - 0s - loss: 4.6069 - acc: 0.0190
Epoch 1/1
 - 0s - loss: 4.6027 - acc: 0.0190
Epoch 1/1
 - 0s - loss: 4.6013 - acc: 0.0190
Epoch 1/1
 - 0s - loss: 4.6030 - acc: 0.0286
Epoch 1/1
 - 0s - loss: 4.5943 - acc: 0.0095
Epoch 1/1
 - 0s - loss: 4.6065 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.5927 - acc: 0.1143
Epoch 1/1
 - 0s - loss: 4.5915 - acc: 0.0476
Epoch 1/1
 - 0s - loss: 4.5987 - acc: 0.0476
Epoch 1/1
 - 0s - loss: 4.5777 - acc: 0.0667
Epoch 1/1
 - 0s - loss: 4.5979 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.5877 - acc: 0.0381
Epoch 1/1
 - 0s - loss: 4.5949 - acc: 0.0286
Epoch 1/1
 - 0s - loss: 4.5996 - acc: 0.0381
Epoch 1/1
 - 0s - loss: 4.6037 - acc: 0.0667
Epoch 1/1
 - 0s - loss: 4.5902 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 4.5902 - acc: 0.0762
Epoch 1/1
 - 0s - loss: 4.5900 - acc: 0.057

Epoch 1/1
 - 0s - loss: 4.1829 - acc: 0.3333
Epoch 1/1
 - 0s - loss: 4.2466 - acc: 0.3714
Epoch 1/1
 - 0s - loss: 4.1771 - acc: 0.3143
Epoch 1/1
 - 0s - loss: 4.2206 - acc: 0.2381
Epoch 1/1
 - 0s - loss: 4.1543 - acc: 0.3429
Epoch 1/1
 - 0s - loss: 4.1623 - acc: 0.3524
Epoch 1/1
 - 0s - loss: 4.0439 - acc: 0.4190
Epoch 1/1
 - 0s - loss: 4.1886 - acc: 0.3238
Epoch 1/1
 - 0s - loss: 4.0955 - acc: 0.3429
Epoch 1/1
 - 0s - loss: 4.1306 - acc: 0.3048
Epoch 1/1
 - 0s - loss: 4.2393 - acc: 0.1810
Epoch 1/1
 - 0s - loss: 4.1298 - acc: 0.4095
Epoch 1/1
 - 0s - loss: 4.0943 - acc: 0.2952
Epoch 1/1
 - 0s - loss: 4.0627 - acc: 0.2476
Epoch 1/1
 - 0s - loss: 4.1005 - acc: 0.3619
Epoch 1/1
 - 0s - loss: 4.1127 - acc: 0.2190
Epoch 1/1
 - 0s - loss: 4.0258 - acc: 0.3714
Epoch 1/1
 - 0s - loss: 4.0535 - acc: 0.3619
Epoch 1/1
 - 0s - loss: 4.0393 - acc: 0.3524
Epoch 1/1
 - 0s - loss: 4.1302 - acc: 0.3905
Epoch 1/1
 - 0s - loss: 4.0213 - acc: 0.3524
Epoch 1/1
 - 0s - loss: 4.1144 - acc: 0.2381
Epoch 1/1


Epoch 1/1
 - 0s - loss: 3.0666 - acc: 0.8667
Epoch 1/1
 - 0s - loss: 3.0774 - acc: 0.8667
Epoch 1/1
 - 0s - loss: 3.0791 - acc: 0.9714
Epoch 1/1
 - 0s - loss: 3.0605 - acc: 0.8762
Epoch 1/1
 - 0s - loss: 3.0183 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 3.0989 - acc: 0.8857
Epoch 1/1
 - 0s - loss: 3.1486 - acc: 0.8095
Epoch 1/1
 - 0s - loss: 2.9471 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 2.9737 - acc: 0.8857
Epoch 1/1
 - 0s - loss: 2.9667 - acc: 0.9238
Epoch 1/1
 - 0s - loss: 3.2427 - acc: 0.7714
Epoch 1/1
 - 0s - loss: 3.1090 - acc: 0.9048
Epoch 1/1
 - 0s - loss: 2.9053 - acc: 0.9619
Epoch 1/1
 - 0s - loss: 2.9819 - acc: 0.8952
Epoch 1/1
 - 0s - loss: 2.9896 - acc: 0.9238
Epoch 1/1
 - 0s - loss: 3.0881 - acc: 0.8762
Epoch 1/1
 - 0s - loss: 2.8962 - acc: 0.9714
Epoch 1/1
 - 0s - loss: 2.7414 - acc: 0.9333
Epoch 1/1
 - 0s - loss: 2.9531 - acc: 0.9143
Epoch 1/1
 - 0s - loss: 2.9753 - acc: 0.9524
Epoch 1/1
 - 0s - loss: 2.8970 - acc: 0.9238
Epoch 1/1
 - 0s - loss: 2.9033 - acc: 0.9143
Epoch 1/1
